In [482]:
# Trace simulation
# This is a notebook that simulates the behaviour of a LRU k-way associative cache
# it is by no means a full simulator. It is strictly designed to calculate the
# hit and miss counts, rates for multi-core cache traces. This simulator
# does not simulate a full cache. Only the hit/miss logic.

## This notebook requires a trace file in the following syntax to work
# The trace file should have a single cache trace in each line of the file
# Each line must follow the following structure:
# [op] 0x[addr] [core]
# [op] is the operation being requested. Either a write (w) or a read (r)
## This simulator does not perform either of those operations. 
## The [op] part is there because the program was used with other simulation
## platforms that are not indicated in this simulator
# [addr] is a hexidecimal representation of the cache address being accessed
# [core] is the ID of the core performing the access
# example trace from a 4-core access pattern to a 4MB LLC cache:
# w 0x00123053052 0
# r 0x0010E0DE0D2 1
# r 0x00122782782 2
# r 0x00122582581 2
# w 0x00123063061 0
# r 0x00122592592 2
# w 0x001151B51B1 3
# w 0x00123073070 0
# r 0x001225A25A4 2
# r 0x00122792791 2
# r 0x001225C25C3 2
# r 0x001225D25D4 2
# r 0x001225E25E1 2
# w 0x00123083081 0
# r 0x00122602602 2
# r 0x000ED8BD8B1 1
# r 0x000FCA6CA62 1
# w 0x001151C51C4 3
# w 0x001230F30F3 0
# r 0x00122612612 2
# r 0x000FCA5CA51 1
# w 0x00123103102 0
# r 0x0010E1CE1C1 1
# w 0x00123113114 0
# w 0x001151D51D0 3
# r 0x001225F25F3 2


In [475]:
import numpy as np
import math
import re

# useful functions
def hex2bin(s,l=0):
    res = str(bin(int(s, 16)))
    res = res[2:]
    res = res.zfill(l)
    return res
def bin2hex(s,l=0):
    res = str(hex(int(s, 2)))
    res = res[2:]
    res = res.zfill(l)
    return res

def bin2dec(s):
    res = int(s, 2)
    return res

In [483]:
def addr_s_t_extract(h_addr,SET_BITS=12,TAG_BITS=24,OFFSET_BITS=4,ADDR_H_LEN=11):
    # chekc if access is first access (mark as all X if an access should be ignored)
    if h_addr == "XXXXXXXXXXX":
        return (0,0)
    b_addr=hex2bin(h_addr,4*ADDR_H_LEN) # get the hex to binary representation
    b_addr_set=b_addr[-(SET_BITS+OFFSET_BITS):-OFFSET_BITS] #get the set bits
    b_addr_tag=b_addr[:-(SET_BITS+OFFSET_BITS)] #get the tag bits
    d_addr_set=bin2dec(b_addr_set) #get the set bits as decimal
    h_addr_tag=bin2hex(b_addr_tag,round(TAG_BITS/4)) #get the tag as hex
    tag_set_tupl=(d_addr_set,h_addr_tag) # create tuple of set and tag
    return tag_set_tupl # return the tuple

In [484]:

def trace_prs(trace_file=""):
    print("Parsing Trace File" + trace_file)
    # open trace file with syntax # [op] 0x[addr] [core]
    with open (trace_file, "r") as tr_f:
        trace_lines = tr_f.read()
    # create the output trace list
    trace_list=[]
    # use the pattern to parse the lines
    pattern="""
    ()
    (?P<op>[a-z]{1})
    (\ 0x)
    (?P<addr>[a-zA-Z0-9]{11})
    (\ )
    (?P<core>[0-9]{1})
    """
    # for every line in the file
    for acss in re.finditer(pattern,trace_lines,re.VERBOSE):
        # get operation (r/w), address (addr), and the core requesting the access
        op=acss['op']
        addr=acss['addr']
        core=acss['core']
        
        # get the addrs set and tag bits
        s_t_tupl = addr_s_t_extract(addr)
        
        # return the access as a dictionary
        trace_dict={
        "set":s_t_tupl[0], # set (integer)
        "tag":s_t_tupl[1], # tag (hex)
        "core":int(core), # core number (integer)
        }
        
        trace_list.append(trace_dict) # add the trace dictionary to a list
    print("FINISHED : Parsing Trace File" + trace_file)
    # return the list of traces when done!
    return trace_list

#tr_ls = trace_prs("simultorTraceFiles/sample_trace_full.trace")
#print(len(tr_ls))

In [485]:
def set_LRU(cache_mem,set_bit,tag,num_ways):
    # if tag is in the list, remove it 
    if tag in cache_mem[set_bit]:
        cache_mem[set_bit].remove(tag)
    
    if len(cache_mem[set_bit]) < num_ways:
        # if list is smalled or equal to num_ways
        # add the new tag
        cache_mem[set_bit].append(tag)
    else:
        # if list is larger than num_ways, remove the
        # LRU, and add new tag
        cache_mem[set_bit].pop(0)
        cache_mem[set_bit].append(tag)
    return cache_mem
def cache_LRU_sim(trace_list,
                  num_ways=16, # number of ways in the cache
                  SET_BITS=12, # number of set bits
                  TAG_BITS=24, # number of tag bits
                  OFFSET_BITS=4, # number of offset bits
                  ADDR_H_LEN=11, # number of hex representing address
                  cache_block_size_B=64, # block size in Bytes
                  num_cores=4, # number of cores accessing the cache
                  num_access=500000 # number of requested traces
                 ): 
    
    # the number of adressable sets in the cache
    NUM_SETS = 2 ** SET_BITS
    
    # the number of all cache lines (blocks) the cache can hold
    NUM_cache_lines = NUM_SETS * num_ways
    
    # the size of the cache, returned as KB and MB
    CACHE_SIZE = NUM_cache_lines * cache_block_size_B
    print("Cache Size = " + str(CACHE_SIZE/1024) + " KB  --> " + str(CACHE_SIZE/1024/1024) + " MB")
    
    # the number of trace instances in the trace file
    print("Number of Trace instances : " + str(len(trace_list)))
    
    # if the trace instances are less than the requested number of accesses,
    # change the requested number of accesses to the number of trace instances
    if len(trace_list)<num_access:
        num_access=len(trace_list)
    
    # intialize the numby 2D array with 0s
    cache_mem = np.zeros(shape=(NUM_SETS,0))
    
    # convert to regular array (I think it is simpler access this way)
    cache_mem = cache_mem.tolist()
    
    # initialize variables required
    hit_c = 0 # total number of hits
    miss_c = 0 # total number of misses
    misses_cores=[] # create a list for all the misses per core count
    hits_cores=[] # create a list for all the hits per core count
    hit_rates_cores=[] # create a list for all the miss rate per core
    miss_rates_cores=[] # create a list for all the hit rate per core
    total_access=0 # counter for total accesses performed
    # initialize the above lists for each core
    for i in range(0,num_cores):
        misses_cores.append(0)
        hits_cores.append(0)
        hit_rates_cores.append(0)
        miss_rates_cores.append(0)
    # start main loop
    for i in trace_list:
        # get set and tag bits
        set_bit = i["set"]
        tag = i["tag"]
        # if tag is in set
        if tag in cache_mem[set_bit]:
            # hit
            hit_c += 1
            # hit for the core
            hits_cores[i["core"]]+= 1
        else:
            # miss
            miss_c += 1
            # miss for the core
            misses_cores[i["core"]]+= 1
       
        # update the LRU list for set_LRU function()
        set_LRU(cache_mem,set_bit,tag,num_ways)
        
        # calculate the hit and miss rates per core
        tot = misses_cores[i["core"]] + hits_cores[i["core"]]
        hit_rates_cores[i["core"]]=hits_cores[i["core"]]/tot
        miss_rates_cores[i["core"]]=misses_cores[i["core"]]/tot
        
        # find total of all accesses performed
        total_access=hit_c+miss_c
        
        #if the number of accesses performed is equal to requested accesses
        # terminate main loop
        if total_access==num_access:
            break
    
    # get total miss and hit rates
    total_hit_rate = hit_c / total_access
    total_miss_rate = miss_c / total_access
    
    # generate output dictionary and return u
    dict_out={
        "Total Accesses":total_access,
        "Total Hits":hit_c,
        "Total Misses":miss_c,
        "Total Hit rate":total_hit_rate,
        "Total Miss rate":total_miss_rate,
        "Hit count per core":hits_cores,
        "Miss count per core":misses_cores,
        "Hit rate per core":hit_rates_cores,
        "Miss rate per core":miss_rates_cores,
        }
    return dict_out

